# Your First Incubator Service

This notebook assumes that the incubator DT is already running.
If not:
1. Please skim through the documentation of the incubator digital twin and pay a special attention to how the containers are set up for communication and time series database. You will see that they follow a similar approach to what has been proposed in the [pre-requisites tutorials](../0-Pre-requisites).
2. Make sure there are no docker containers running, to avoid clashes with the containers used by the incubator DT.
3. Follow the instructions in the [incubator_dt repository README](../incubator_dt/README.md) (section `Running the Digital Twin`) to get it started (by means of running the [start_all_services.py](../incubator_dt/software/startup/start_all_services.py) script), with the following adjustments:
   1. If you wish, you can reuse the virtual environment you have possibly already created in the current repository.
   2. **Note** that the initial setup may take some time, as the docker images are being built.

After the incubator DT is running, you should be able to see an output like the following:
````
time           execution_interval  elapsed  heater_on  fan_on   room   box_air_temperature  state 
19/11 16:17:59  3.00                0.01     True       False   10.70  19.68                Heating
19/11 16:18:02  3.00                0.03     True       True    10.70  19.57                Heating
19/11 16:18:05  3.00                0.01     True       True    10.70  19.57                Heating
19/11 16:18:08  3.00                0.01     True       True    10.69  19.47                Heating
19/11 16:18:11  3.00                0.01     True       True    10.69  19.41                Heating
````

If that's the case, continue running this notebook.

## What is a Digital Twin (DT) Service?

In the context of the incubator digital twin, a service is a process that communicates via a [RabbitMQ message exchange](https://www.rabbitmq.com/), running within a [Docker](https://www.docker.com/products/docker-desktop) container. 

### Types of DT Services

Currently, there are two types of services:

1. **Server Services**  
   These services operate similarly to a client-server model. They perform actions only in response to RabbitMQ messages that encapsulate requests for specific computations. Once the computation is completed, the result is sent back as a RabbitMQ message in response to the original request.

2. **Reactive Services**  
   These services subscribe to a particular RabbitMQ topic. When a message arrives on the subscribed topic, they perform a computation and publish the result to another RabbitMQ topic. Other services can then subscribe to the topic where the result is published, allowing a chain of computations across services.

### Example Services

The following examples demonstrate one of each service type, including a mixed service that both acts as a client server, but also has ongoing computation as a reactive service. The examples also make the services communicate with each other and the incubator physical twin. Altogether these services give you everything you need to build your own digital twin services using rabbitmq.

- **AverageService**: This is a server service that computes the average of a given list of values.
- **MovingAverageTemperatureService**: This is a hybrid service that maintains a moving average of the last N values of the temperature in the incubator. It communicates with the average service to get the average. In addition, as a server, it responds to requests to reset the moving average.
- **PTReconfigurationService**: This is a reactive service that reconfigures the incubator physical twin whenever the difference between the temperature moving average and the actual temperature is high. It uses the values of the `MovingAverageTemperatureService`.

### Configuration

Most services use the configuration file located at the [startup.conf](../incubator_dt/software/startup.conf) of the incubator DT project for parameters such as RabbitMQ connection details and other settings.

## Setting up dependencies

First we're going to set up the PYTHONPATH so that we can load the python code from the [incubator DT repository](../incubator_dt/software/).

In [1]:
# Configure python path to load incubator modules
import sys
import os

# Get the current working directory. Should be 1-Incubator-Service
current_dir = os.getcwd()

assert os.path.basename(current_dir) == '1-Incubator-Service', 'Current directory is not 1-Incubator-Service'

# Get the parent directory. Should be the root of the repository
parent_dir = os.path.dirname(current_dir)

# The root of the repo should contain the incubator_dt folder. Otherwise something went wrong in 0-Pre-requisites.
assert os.path.exists(os.path.join(parent_dir, 'incubator_dt')), 'incubator_dt folder not found in the repository root'

incubator_dt_software_dir = os.path.join(parent_dir, 'incubator_dt', 'software')

assert os.path.exists(incubator_dt_software_dir), 'incubator_dt software directory not found'

# Add the parent directory to sys.path
sys.path.append(incubator_dt_software_dir)

In [2]:
# After the above, we should be able to import incubator modules

# The following imports a class that makes it easier to interact with the RabbitMQ server, just to test the above code.
from incubator.communication.server.rabbitmq import Rabbitmq

## Logging configuration for services

Each service will use the standard logging framework from Python. We use a single logging configuration file for all services, and adjust it as needed. This makes it easier to quickly configure the logging for the different services we will create.

The logging configuration in the next cell defines at least two loggers (`root` and `AverageService`), two handlers (`consoleHandler` and `AverageServiceFileHandler`), and one formatter (`simpleFormatter`). Here's a detailed explanation of what each section does:

### 1. **Loggers**
Loggers are components that generate log messages. This configuration defines at least two loggers:
- `root`: This is the default logger that all other loggers inherit from unless otherwise specified. It will capture log messages from the entire application.
- `AverageService`: This is a specific logger named `AverageService`, which logs messages related to the `AverageService` component.

### 2. **Handlers**
Handlers are responsible for deciding where the log messages go (e.g., to a file, console, etc.). This configuration defines at least two handlers:
- `consoleHandler`: Sends log messages to the console (or terminal) using `sys.stdout`.
- `AverageServiceFileHandler`: Sends log messages from the `AverageService` logger to a file named `AverageService.log`. It opens the file in write mode (`'w'`), meaning it will overwrite the file each time the logger is used.

### 3. **Formatters**
The formatter defines the format of the log messages:
- **`simpleFormatter`**: This formatter formats log messages with the following pattern:  
  `%(asctime)s.%(msecs)03d %(levelname)s %(name)s : %(message)s`  
  - `%(asctime)s`: Timestamp of the log event.
  - `%(msecs)03d`: Milliseconds portion of the time, padded to 3 digits.
  - `%(levelname)s`: The log level (e.g., DEBUG, INFO).
  - `%(name)s`: The name of the logger (e.g., `AverageService`).
  - `%(message)s`: The actual log message.
  
  The `datefmt` specifies the date format as `'%Y-%m-%d %H:%M:%S'`.

### 4. **Logging Configuration Details**

#### **[loggers]**
```ini
[loggers]
keys=root,AverageService
```
- This section defines at least two loggers, `root` and `AverageService`.

#### **[logger_root]**
```ini
[logger_root]
level=DEBUG
handlers=consoleHandler
```
- The `root` logger is set to capture log messages at the `DEBUG` level or higher. The `consoleHandler` is used to send these log messages to the console.

#### **[logger_AverageService]**
```ini
[logger_AverageService]
level=DEBUG
handlers=AverageServiceFileHandler
qualname=AverageService
propagate=0
```
- This logger captures log messages for the `AverageService` component, at the `DEBUG` level or higher, and sends them to the `AverageService.log` file.
- `qualname=AverageService`: This logger is used when you call `logging.getLogger('AverageService')` in your code.
- `propagate=0`: This prevents log messages from being passed to the parent logger (i.e., the `root` logger), meaning logs from `AverageService` will not appear in the console.

#### **[handlers]**
```ini
[handlers]
keys=consoleHandler,AverageServiceFileHandler
```
- This defines at least two handlers: one for the console and one for the file.

#### **[handler_consoleHandler]**
```ini
[handler_consoleHandler]
class=StreamHandler
formatter=simpleFormatter
args=(sys.stdout,)
```
- This handler sends log messages to the console (standard output).
- It uses the `simpleFormatter` to format the log messages.

#### **[handler_AverageServiceFileHandler]**
```ini
[handler_AverageServiceFileHandler]
class=FileHandler
formatter=simpleFormatter
args=('AverageService.log', 'w')
```
- This handler sends log messages to the `AverageService.log` file.
- The file is opened in write mode (`'w'`), meaning the contents are overwritten each time logging occurs.
- It also uses the `simpleFormatter` for formatting log messages.

### 5. **How It Works in Practice**

- **Root Logger**: Any logs from the rest of your application that don't specify a logger will be handled by the `root` logger. These logs will appear in the console.
- **AverageService Logger**: Logs specifically generated by `logging.getLogger('AverageService')` will be written to the `AverageService.log` file. These logs will not propagate to the `root` logger, meaning they won’t appear in the console.

### Example Usage:

```python
import logging
import logging.config

# Load the logging configuration
logging.config.fileConfig('logging.conf')

# Get the AverageService logger
average_service_logger = logging.getLogger('AverageService')

# Log a message using AverageService logger
average_service_logger.debug("This is a debug message for AverageService.")

# Log a message using the root logger
logging.debug("This is a debug message for root.")
```

- **Message for `AverageService`**: Will be logged in `AverageService.log`.
- **Message for `root`**: Will appear in the console.

In [3]:
%%writefile logging.conf
[loggers]
keys=root,AverageService

[handlers]
keys=consoleHandler,AverageServiceFileHandler

[formatters]
keys=simpleFormatter

[logger_root]
level=DEBUG
handlers=consoleHandler

[logger_AverageService]
level=DEBUG
handlers=AverageServiceFileHandler
qualname=AverageService
propagate=0

[handler_consoleHandler]
class=StreamHandler
formatter=simpleFormatter
args=(sys.stdout,)

[handler_AverageServiceFileHandler]
class=FileHandler
formatter=simpleFormatter
args=('AverageService.log', 'w')

[formatter_simpleFormatter]
format=%(asctime)s.%(msecs)03d %(levelname)s %(name)s : %(message)s
datefmt=%Y-%m-%d %H:%M:%S

Overwriting logging.conf


## Connection configuration for services

In addition, all services will use the configuration from the startup.conf file, available inside the incubator repository:

In [4]:
# Print contents of startup.conf file
startup_conf = os.path.join(os.path.dirname(os.getcwd()), 'incubator_dt', 'software','startup.conf')
assert os.path.exists(startup_conf), 'startup.conf file not found'
with open(startup_conf, 'r') as f:
    print(f.read())


rabbitmq: {
    ip = "localhost"
    port = 5672
    username = incubator
    password = incubator
    exchange = Incubator_AMQP
    type = topic
    vhost = /
    # ssl: {   # Enable for ssl support. Only works if the RabbitMQ server is configured to support it.
    #     protocol: "PROTOCOL_TLS",
    #     ciphers : "ECDHE+AESGCM:!ECDSA"
    # }
}
influxdb: {
    url = http://localhost:8086
    token = "-g7q1xIvZqY8BA82zC7uMmJS1zeTj61SQjDCY40DkY6IpPBpvna2YoQPdSeENiekgVLMd91xA95smSkhhbtO7Q=="
    org = incubator
    bucket = incubator
}
physical_twin: {
    controller: {
        temperature_desired = 35.0,
        lower_bound = 5.0,
        heating_time = 20.0,
        heating_gap = 30.0
    }
    controller_open_loop: {
        n_samples_period = 40,
        n_samples_heating = 5,
    }
}
digital_twin: {
    models: {
        plant: {
            param4: {
                C_air = 267.55929458,
                G_box = 0.5763498,
                C_heater = 329.25376821,
               

## AverageService

Since each service runs in a separate python process, we will write the python code to a file, and run it from there.

In [5]:
%%writefile averageservice.py

# Configure python path to load incubator modules
import sys
import os
import logging
import logging.config
import time

# Get the current working directory. Should be 1-Incubator-Service
current_dir = os.getcwd()

assert os.path.basename(current_dir) == '1-Incubator-Service', 'Current directory is not 1-Incubator-Service'

# Get the parent directory. Should be the root of the repository
parent_dir = os.path.dirname(current_dir)

# The root of the repo should contain the incubator_dt folder. Otherwise something went wrong in 0-Pre-requisites.
assert os.path.exists(os.path.join(parent_dir, 'incubator_dt')), 'incubator_dt folder not found in the repository root'

incubator_dt_software_dir = os.path.join(parent_dir, 'incubator_dt', 'software')

assert os.path.exists(incubator_dt_software_dir), 'incubator_dt software directory not found'

# Add the parent directory to sys.path
sys.path.append(incubator_dt_software_dir)

from incubator.communication.server.rpc_server import RPCServer

class AverageService(RPCServer):
    """
    This is a server service that computes the average of a given list of values.
    It extends the RPCServer class, which is a class that listens to a RabbitMQ queue and waits for messages to arrive, and hides much of the complexity of the server service. 
    All we need to do to implement the average service is implement a method called "compute_average" that takes a list of values and returns the average of those values. This method will be called by the RPCServer class when a message arrives in the RabbitMQ queue containing the name of the method to call and the arguments to pass to the method.
    """
    def __init__(self, rabbitmq_config):
        super().__init__(**rabbitmq_config)
        self._l = logging.getLogger("AverageService")

    def setup(self):
        """ 
        Setup the RabbitMQ connection and declare the routing_key (this is the topic that this server will listen to) and queue (the name of the queue where all messages addressed to routing_key will be placed in by the RabbitMQ server).

        We use the same name for both the routing_key and the queue name. This is not necessary, but it makes it easier to understand what is happening in the RabbitMQ server.        
        """
        super(AverageService, self).setup(routing_key='dtcourse.incubator.averageservice', queue_name='dtcourse.incubator.averageservice')

        self._l.info(f"AverageService setup complete.")

    def compute_average(self, values, reply_fun):
        """ 
        This is the method that will be invoked by the RPCServer class when a message arrives in the RabbitMQ queue. The reply_fun is a function that we can call to send the results back to the client that sent the message.
        """
        average = 0.0

        # Log the values received.
        self._l.info(f"compute_average called. Received values: {values}")

        # Compute the average of the values.
        if len(values) > 0:
            average = sum(values) / len(values)
        else:
            self._l.warning("Received an empty list of values. Cannot compute average. Returning error")
            reply_fun({"error": "Received an empty list of values. Cannot compute average."})
            return

        # Prepare the results to send back.
        result_msg = {
            "average": average
        }

        # Send results back.
        reply_fun(result_msg)
    
if __name__ == "__main__":
    # Get utility functions to config logging and load configuration
    from incubator.config.config import load_config
    from pyhocon import ConfigFactory

    # Get logging configuration
    logging.config.fileConfig("logging.conf")

    # Get path to the startup.conf file used in the incubator dt:
    startup_conf = os.path.join(os.path.dirname(os.getcwd()), 'incubator_dt', 'software','startup.conf')
    assert os.path.exists(startup_conf), 'startup.conf file not found'

    # The startup.conf comes from the incubator dt repository.
    config = ConfigFactory.parse_file(startup_conf)
    service = AverageService(rabbitmq_config=config["rabbitmq"])

    service.setup()
    
    # Start the AverageService
    service.start_serving()

Overwriting averageservice.py


In [6]:
import subprocess
import time

# Start a process asynchronously
avg_service_proc = subprocess.Popen(["python", "averageservice.py"])

# Wait for 5 seconds for the process to start
time.sleep(5)

# Print the PID of the process. You can search for this in your task manager to see the process running and kill it if necessary.
print(avg_service_proc.pid)

9628


In [7]:
# Print contents of log file to verify that the service is running
with open('AverageService.log', 'r') as f:
    print(f.read())


2024-10-18 11:09:59.031 DEBUG AverageService : Ready to listen for msgs in queue dtcourse.incubator.averageservice bound to topic dtcourse.incubator.averageservice
2024-10-18 11:09:59.031 INFO AverageService : AverageService setup complete.



Now that the service is running let us send a request message to execute an operation.

In [8]:
# Import RPCClient class from incubator, which makes connecting to RabbitMQ and calling remote methods easy.
from incubator.communication.server.rpc_client import RPCClient
from pyhocon import ConfigFactory

# Get config
startup_conf = os.path.join(os.path.dirname(os.getcwd()), 'incubator_dt', 'software','startup.conf')
config = ConfigFactory.parse_file(startup_conf)

with RPCClient(**(config["rabbitmq"])) as client:
    reply = client.invoke_method("dtcourse.incubator.averageservice", "compute_average", {"values": [1.0, 2.0, 3.0, 4.0, 5.0]})
    print(reply)
    assert 2.9 < reply["average"] < 3.1, "Average is not 3.0"

{'average': 3.0}


In [9]:
# Now send something non sensical, to get an error:
with RPCClient(**(config["rabbitmq"])) as client:
    reply = client.invoke_method("dtcourse.incubator.averageservice", "compute_average", {"values": []})
    print(reply)
    assert "error" in reply, "Error not received"


{'error': 'Received an empty list of values. Cannot compute average.'}


In [10]:
with RPCClient(**(config["rabbitmq"])) as client:
    reply = client.invoke_method("dtcourse.incubator.averageservice", "some_other_method", {"values": []})
    print(reply)
    assert "error" in reply, "Error not received"

{'error': 'Method specified does not exist: some_other_method.'}


In [11]:
# Print contents of log file to check that the service received the request and sent a response
with open('AverageService.log', 'r') as f:
    print(f.read())

2024-10-18 11:09:59.031 DEBUG AverageService : Ready to listen for msgs in queue dtcourse.incubator.averageservice bound to topic dtcourse.incubator.averageservice
2024-10-18 11:09:59.031 INFO AverageService : AverageService setup complete.
2024-10-18 11:10:03.945 DEBUG AverageService : Message received: 
f{'method': 'compute_average', 'args': {'values': [1.0, 2.0, 3.0, 4.0, 5.0]}}
2024-10-18 11:10:03.945 DEBUG AverageService : routing_key_reply = famq.gen-eiACXY4Rqag5SeIa29VjSQ
2024-10-18 11:10:03.945 DEBUG AverageService : request_id = f891a627a-b37f-4ab0-8d0f-a3898db7e0c8
2024-10-18 11:10:03.945 INFO AverageService : compute_average called. Received values: [1.0, 2.0, 3.0, 4.0, 5.0]
2024-10-18 11:10:03.945 DEBUG AverageService : Sending reply msg:
{'average': 3.0}
2024-10-18 11:10:04.125 DEBUG AverageService : Message received: 
f{'method': 'compute_average', 'args': {'values': []}}
2024-10-18 11:10:04.125 DEBUG AverageService : routing_key_reply = famq.gen-BK1fil2N-3NUkB9nkaQ0Lg
20

In [12]:
# Terminate process forcefully. This is the simplest and most readable way to terminate services, as most resources are automatically cleaned by the rabbitmq classes. Not recommended for a production environment though.

avg_service_proc.terminate() # Terminate the process. Send SIGTERM signal to the process. The process can catch this signal and perform cleanup operations before exiting, so we wait.
avg_service_proc.wait()  # Wait for the process to exit fully. Should print exit code 1, due to interruption.

# Ensure process has exited
assert avg_service_proc.returncode is not None, 'Process has not exited'